## Load Utilities
Load all the utilities we need, ``utilities.sage`` includes most of the utilities we need (the dual low pass solve, functions to 

In [1]:
import sage.libs.ecl
sage.libs.ecl.ecl_eval("(ext:set-limit 'ext:heap-size 0)")

load('utility.sage')
load('cfe.sage')

In [4]:
L = matrix(SR, [
        [-1,1,1],
        [1,-1,1],
        [1,1,-1]
    ])

Lt = L.transpose().inverse()

pi_ = [
    vector([0,0,0]),
    vector([pi,0,0]),
    vector([0,pi,0]),
    vector([pi,pi,0]),
    vector([0, 0,pi]),
    vector([pi,0,pi]),
    vector([0,pi,pi]),
    vector([pi,pi,pi]),
]

pi_l = [Lt*p for p in pi_]

bases = [
    (1, vector([2,0,0])),
    (1, vector([0,2,0])),
    (1, vector([0,0,2])),
    (1/2, vector([-1,1,1])),
    (1/2, vector([1,-1,1])),
    (1/2, vector([1,1,-1])),
    (1/2, vector([-1,-1,-1])),
]



# Setup the inital dual, with an empty low pass
md = [[]] + [extend_along(2, w, _) for w,_ in bases]

# 
M_D = [filter_to_zdomain(_, L) for _ in md]
Z = [var('z_0'), var('z_1'), var('z_2')]

In [5]:
D = matrix([[m.subs({z:(w/z) for (w, z) in zip(expl(-I*p), Z)}) for m in M_D[1:]] for p in pi_[1:]])
M_0 = D.det()  *(1/(z_0*z_1*z_2))^2

In [6]:
M_0.expand().factor()
LP = zdomain_to_filter(M_0, 3, L)
points([p for p,_ in LP])
c = CFE(LP)

In [7]:
c.plot()

Graphics3d Object

In [9]:
p_,q_ = 1, 1

In [10]:

# This is hard coded for this case, 
# it works out if you calculate the DET

D = -2

f = c
cq = c
for i in xrange(1,q_):
    cq = cq*c
    
for i in xrange(1,q_ + 1):
    f = f*c

f = find_dual_lp(f)

m_0d = cq * f
md[0] = m_0d.filter

creating ring..
giving it symmetry...
making filter..
Solving...
[c_0, c_1]
No solution, increasing filter ring
creating ring..
giving it symmetry...
making filter..
Solving...
[c_0, c_1, c_2]
Got solution


In [11]:
M_D = [filter_to_zdomain(_, L) for _ in md]
A = matrix([[m.subs({z:(w/z) for (w, z) in zip(expl(-I*p), Z)}) for m in M_D[0:]] for p in pi_[0:]])

In [12]:
def solve_for_dual(idx, A):
    __ = idx
    S = matrix([
        [A[i,j] for j in xrange(0,8) if j != __]
    for i in xrange(0,8) if i != 0])
    return S.det()*((-1)**__)

In [13]:
# Solve for all the duals
m_0 = solve_for_dual(0, A)
m_1 = solve_for_dual(1, A)
m_2 = solve_for_dual(2, A)
m_3 = solve_for_dual(3, A)
m_4 = solve_for_dual(4, A)
m_5 = solve_for_dual(5, A)
m_6 = solve_for_dual(6, A)
m_7 = solve_for_dual(7, A)

In [14]:
# Normalize the family
m_0f = CFE(zdomain_to_filter(m_0, 3, L))
D = taylor(filter_dft(m_0f.filter), [w_0, w_1, w_2], 0,0)
print D

m = [c, 
    CFE(zdomain_to_filter(m_1* (1/D), 3, L)),
    CFE(zdomain_to_filter(m_2* (1/D), 3, L)), 
    CFE(zdomain_to_filter(m_3* (1/D), 3, L)), 
    CFE(zdomain_to_filter(m_4*(1/D), 3, L)), 
    CFE(zdomain_to_filter(m_5 *(1/D), 3, L)), 
    CFE(zdomain_to_filter(m_6*(1/D), 3, L)), 
    CFE(zdomain_to_filter(m_7*(1/D), 3, L))]

2


In [15]:
#Setup for the next stage of boot strapping

m[0] = CFE(md[0])

md[1] = m[1]
md[2] = m[2]
md[3] = m[3]
md[4] = m[4]
md[5] = m[5]
md[6] = m[6]
md[7] = m[7]

In [16]:
# Next stage, swap filters and find dual
g = c
cp = c
for i in xrange(1, p_):
    cp = cp * c

    
g = find_dual_lp(m[0]*cp,2)

md[0] = g * cp

creating ring..
giving it symmetry...
making filter..
Solving...
[c_0, c_1]
Got solution


In [17]:
M_D = [filter_to_zdomain(_.filter, L) for _ in md]
A = matrix([[m.subs({z:(w/z) for (w, z) in zip(expl(-I*p), Z)}) for m in M_D[0:]] for p in pi_[0:]])

In [18]:
# We need the constant factor, so we manually solve for this guy, and 
# Do a taylor expansion to grab D, we should technically know m_0a already 
m_0a = solve_for_dual(0, A)

In [19]:
m_0af = CFE(zdomain_to_filter(m_0a, 3, L))

In [ ]:
D = taylor(filter_dft(m_0af.filter), [w_0, w_1, w_2], 0, 0)

In [ ]:
m_1a = solve_for_dual(1, A)
m_2a = solve_for_dual(2, A)
m_3a = solve_for_dual(3, A)
m_4a = solve_for_dual(4, A)
m_5a = solve_for_dual(5, A)
m_6a = solve_for_dual(6, A)
m_7a = solve_for_dual(7, A)